In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import time
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import seaborn as sns

### Global variables

In [ ]:
data_folder = 'data/'
training_data = data_folder + 'train.csv'
testing_data = data_folder + 'test.csv'

methods = ['mse', 'mae', 'log', 'ridge-regression']
method = methods[3]

pos_weight = 1
lambda_1 = 3

# Load the data

We map the 'b' and 's' labels to 0 and 1 respectively.

In [ ]:
def load_data():
    Y = np.genfromtxt(training_data, delimiter=',', dtype=None, skip_header=1, usecols=[1], converters={1: lambda x: 0 if b'b'==x else 1})
    
    data = np.genfromtxt(training_data, delimiter=',', skip_header=1)
    X = data[:, 2:]
    
    return X, Y

In [ ]:
X, Y = load_data()

## Clean and standardize the features

### Counting the number of invalid datapoints per column

In [ ]:
invalids = np.count_nonzero(X == -999, axis=0)
print(invalids)

Therefore we get rid of columns [0,4,5,6,12,23,24,25,26,27,28]

In [ ]:
def clean_and_standardize_features(X):
    X_clean = np.delete(X,[0,4,5,6,12,23,24,25,26,27,28], axis=1)
    X_standardized = (X_clean - X_clean.mean(axis=0))/X_clean.std(axis = 0)
    X_standardized = np.insert(X_standardized, 0, 1, axis=1)
    return X_standardized

In [ ]:
X_standardized = clean_and_standardize_features(X)
N = X_standardized.shape[1]

In [ ]:
correlation_matrix = np.corrcoef(X_standardized[:10000,:], rowvar=False)
suffix = time.time()
np.savetxt('correlation_matrix_' + str(suffix) + '.csv', correlation_matrix, fmt="%0.2f", delimiter=",", comments='')

In [ ]:
def correlation_heatmap(train):
    correlations = np.corrcoef(X_standardized[:15000,:], rowvar=False)

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show();
    
correlation_heatmap(X_standardized)

## Methods from lab1 and lab2

In [ ]:
def compute_loss(y, tx, w):
    predictions = tx@w
    error = y-predictions
    if(method == 'mse'):
        return 1/(2*y.shape[0])*np.sum(error*error)
    elif(method == 'mae'):
        return 1/(2*y.shape[0])*np.sum(np.abs(error))
    elif(method == 'log'):
        predictions = predict(tx, w)
        return -np.sum(y*np.log(predictions)*pos_weight + (1-y)*np.log(1-predictions))/y.shape[0]
    elif(method == 'ridge-regression'):
        return 1/(2*y.shape[0])*np.sum(error*error)+ lambda_1*np.linalg.norm(w)**2

In [ ]:
def grid_search(y, tx, w0, w1):
    losses = np.zeros((len(w0), len(w1)))
    for i in range(len(w0)):
        for j in range(len(w1)):
            losses[i][j] = compute_loss(y, tx, np.array([w0[i], w1[j]]))
    return losses

In [ ]:
def compute_gradient(y, tx, w):
    if(method=='log'):
        prediction = predict(tx, w)
    else:
        prediction = tx@w
    error = y-prediction
    gradient = -1/y.shape[0]*tx.T@error
    return gradient

In [ ]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    i = 0
    initial_w = initial_w/100 # initialize at small weights else gradient descent might explode at first iteration
    w_res = initial_w
    loss_hist = []
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        w = w - gamma * gradient
        # store w and loss
        w_res = w
        loss_hist.append(loss)
        #print("Gradient Descent({bi}/{ti}): loss={l}".format(
              #bi=n_iter, ti=max_iters - 1, l=loss))
        # Log Progress
        i = i + 1
        if i % 1000 == 0:
            print("iter: " + str(i) + " loss: "+str(loss_hist[-1]))

    return loss_hist, w_res

In [ ]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    w_res = initial_w
    loss_res = 0
    w = initial_w
    ti = max_iters
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32, max_iters):
        gradient = compute_gradient(minibatch_y, minibatch_tx, w)
        loss = compute_loss(minibatch_y, minibatch_tx, w)
        w = w- gamma * gradient
        # store w and loss
        w_res = w
        loss_res = loss
    return loss_res, w_res

We decided to use the sigmoid function: $$S(z) = \frac{1}{1 + e^{-z}}$$ <br />
to map the predicted values to probabilities of the event being a signal(1) rather than background(0) 

In [ ]:
def sigmoid(z):
    return 1 / (1 + math.exp(-z))

In [ ]:
sigmoid_vec = np.vectorize(sigmoid)

In [ ]:
def predict(x, w):
    temp = x@w
    return sigmoid_vec(temp)

We use the cross-entropy cost function for loss computation: 
$$J(\theta) = -\frac{1}{N} * (y^T log(sigmoid(Xw)) + (1-y)^T log(1-sigmoid(Xw)))$$

In [ ]:
# find where sigmoid goes to 1

max_iters = 200
gamma = 0.3
batch_size = 1

method = methods[2]
losses, w = gradient_descent(Y, X_standardized, np.random.random(X_standardized.shape[1])/100, max_iters, gamma)
print("w : ")
print(w)
plt.plot(losses)

### Additional methods

We label the results <0.5 to -1, and the rest to 1

In [ ]:
#set the threshold to 0.618 for mse, 0.458 for cross-entropy
def label_results(Y_predicted, threshold=0.61):
    f = lambda x: -1 if x<threshold else 1
    f_vec = np.vectorize(f)
    return f_vec(Y_predicted)

In [ ]:
def performance(Y, Y_predicted):
    return np.sum(Y == Y_predicted)/Y.shape[0]

In [ ]:
def evaluate_performance(Y, Y_predicted):
    false_negatives = 0
    true_negatives = 0
    false_positives = 0
    true_positives = 0
    for i in range(Y.shape[0]):
        if(Y[i]==-1):
            if(Y_predicted[i]==-1):
                true_negatives += 1
            else:
                false_positives += 1
        else:
            if(Y_predicted[i]==-1):
                false_negatives += 1
            else:
                true_positives += 1
    #print('True positives: ' + str(true_positives))
    #print('False positives: ' + str(false_positives))
    #print('True negatives: ' + str(true_negatives))
    #print('False negatives: ' + str(false_negatives))

In [ ]:
def find_best_threshold(Y_predicted, Y_labeled):
    best_t = 0.5
    best_perf = performance(Y_labeled, label_results(Y_predicted, threshold=best_t))
    for i in range(0, 100):
        perf = performance(Y_labeled, label_results(Y_predicted, threshold=i/100))
        #print('threshold is: ' + str(i/100))
        #print('performance is: ' + str(perf))
        if(perf>best_perf):
            best_t = i/100
            best_perf = perf
    print('best threshold is: ' + str(best_t))
    print('best performance is: ' + str(best_perf))
    return best_t, best_perf

## Trying the $L_2$ - regularization

In [ ]:
lambda_prime = lambda_1 * 2 * N
temp = np.linalg.inv(X_standardized.T@X_standardized + np.identity(N)*lambda_prime)
temp_2 = temp@X_standardized.T
w_L_2 = temp_2@Y

In [ ]:
Y_predicted = predict(X_standardized, w_L_2)
Y_labeled = label_results(Y_predicted)
print(performance(Y_labeled, label_results(Y)))
# 0.735 for mse
#0.74038 for cross-entropy
evaluate_performance(label_results(Y), Y_labeled)

In [ ]:
find_best_threshold(Y_predicted, label_results(Y))

## Trying the ridge regression

In [ ]:
max_iters = 1000
gamma = 0.15
batch_size = 1

method = methods[3]
losses, w_ridge = gradient_descent(Y, X_standardized, np.ones(X_standardized.shape[1]), max_iters, gamma)
print(w_ridge)
plt.plot(losses)

In [ ]:
Y_predicted = predict(X_standardized, w_ridge)
Y_labeled = label_results(Y_predicted)
print(performance(Y_labeled, label_results(Y)))
# 0.735 for mse
#0.74038 for cross-entropy
evaluate_performance(label_results(Y), Y_labeled)

In [ ]:
find_best_threshold(Y_predicted, label_results(Y))

## Trying gradient descent

In [ ]:
max_iters = 100
gamma = 1
batch_size = 1

method = methods[2]
losses, w = gradient_descent(Y, X_standardized, np.random.random(X_standardized.shape[1]), max_iters, gamma)
print(w)
plt.plot(losses)

In [ ]:
# compare different values of gamma for gradient descent
max_iters = 600
gamma = [0.5, 0.1, 0.15, 0.2, 0.3, 1]
batch_size = 1
whichLoss = 100
which = 100
losses = np.ndarray([len(gamma),1])
w = np.ndarray([len(gamma), X_standardized.shape[1]])
for i in range(len(gamma)):
    loss, wi = gradient_descent(Y, X_standardized, np.random.random(X_standardized.shape[1]), max_iters, gamma[i]) 
    plt.tight_layout()
    plt.subplot(int(str(32)+str(i+1)))
    plt.plot(loss)
    plt.xlabel("# of iterations")
    plt.ylabel("Cost")
    plt.title("Gamma = " + str(gamma[i]))
    print(loss[-1])
    if(loss[-1] < whichLoss):
        whichLoss = loss[-1]
        which = i
    losses[i] = loss[-1]
    w[i,:] = wi

print("smallest cost:" + str(losses[which]) + "at gamma=" + str(gamma[which]))


In [ ]:
try:
    Y_predicted = predict(X_standardized, w)
except:
    Y_predicted = predict(X_standardized, w[which,:])
Y_labeled = label_results(Y_predicted, 0.46)
print(performance(Y_labeled, label_results(Y)))
# 0.735 for mse
#0.74038 for cross-entropy

In [ ]:
evaluate_performance(label_results(Y), Y_labeled)

In [ ]:
find_best_threshold(Y_predicted, label_results(Y))

# Oversampling the signal class

In [ ]:
all_data = np.column_stack([Y, X_standardized])
distrib = np.bincount(all_data[:,0].astype(int))
prob = 1/distrib[all_data[:, 0].astype(int)].astype(float)
prob /= prob.sum()
all_data = all_data[np.random.choice(np.arange(len(all_data)), size=np.count_nonzero(distrib)*distrib.max(), p=prob)]
print(all_data.shape)

### Rerunning the gradient descent with oversampled data

In [ ]:
max_iters = 500
gamma = 0.15
batch_size = 1

losses, w = gradient_descent(all_data[:,0], all_data[:,1:], np.ones(X_standardized.shape[1]), max_iters, gamma)
print(w)
print(losses[-1])
plt.plot(losses)

In [ ]:
Y_predicted = predict(X_standardized, w)
Y_labeled = label_results(Y_predicted)
print(performance(Y_labeled, label_results(Y)))
# 0.735 for mse
#0.74038 for cross-entropy
evaluate_performance(label_results(Y), Y_labeled)

In [ ]:
find_best_threshold(Y_predicted, label_results(Y))

## Load test data

In [ ]:
test_data = np.genfromtxt(testing_data, delimiter=',', skip_header=1)
test_X = test_data[:, 2:]
test_X_standardized = clean_and_standardize_features(test_X)

In [ ]:
try:
    test_predictions = label_results(predict(test_X_standardized, w))
except:
    test_predictions = label_results(predict(test_X_standardized, w[which]), 0.46)
test_ids = range(350000,918238)

In [ ]:
test_results = np.column_stack([test_ids, test_predictions])
suffix = time.time()
np.savetxt('submission' + str(suffix) + '.csv', test_results, fmt="%d", delimiter=",", header="Id,Prediction", comments='')